In [1]:
import pandas as pd
from tqdm import tqdm

In [6]:
# inputs
fn_metadata = '../data/intermediary/element_metadata_15k.jsonl.gz'
fn_trends = '../data/input/trending_searches_by_category.csv'

In [7]:
df = pd.read_json(fn_metadata, 
                  lines=True)

trends = pd.read_csv(fn_trends)

In [8]:
df.head(2)

,text,link,domain,xpath,category,tag,attrs,dimensions,location,area_page,...,fn_metadata,q1_area,q2_area,q3_area,q4_area,q5_area,label,category_standard,date_parsed,date_collected
0,testicular cancer,None,google.com,/html/body/div[10]/div/div[6]/div/div[4]/div/d...,answer-expand,div,"{'jsname': 'bVEB4e', 'class': ['QRY00e'], 'rol...","{'height': 48.0, 'width': 315.0}","{'x': 24, 'y': 3220}",15120.0,...,../data/intermediary/google_search/iPhone-X/20...,0.0,0.0,0.0,0.0,15120.0,answer,google-expandable-answer,2020-03-21,2019-11-28
1,testicular torsion,None,google.com,/html/body/div[10]/div/div[6]/div/div[4]/div/d...,answer-expand,div,"{'jsname': 'bVEB4e', 'class': ['QRY00e'], 'rol...","{'height': 48.0, 'width': 315.0}","{'x': 24, 'y': 3269}",15120.0,...,../data/intermediary/google_search/iPhone-X/20...,0.0,0.0,0.0,0.0,15120.0,answer,google-expandable-answer,2020-03-21,2019-11-28


In [9]:
df.loc[:, 'search'] = df.fn_input.apply(lambda x : ('').join(x.split('/html')[0].split('/')[8:]))

In [10]:
trends.loc[:, 'search'] = trends['search'].str.replace(' ', '-')

In [11]:
trends.search.nunique()

11648

In [12]:
df['search'].iloc[0]

'Swollen-testicle'

In [13]:
cat2searches = {}
for cat in trends.category.unique():
    searches = (trends[trends.category == cat].search.tolist())
    cat2searches[cat] = searches

In [14]:
cat2desc = {
    'b' : 'Business',
    'e' : 'Entertainment',
    'm' : 'Health',
    't' : 'Science and Tech',
    's' : 'Sports',
    'h' : 'Top Stories',
    'real' : 'User Searches',
    'mix' : 'Multiple Categories'
}

In [15]:
import random
from scipy.stats import t

In [17]:
cat2df = {}
for k, v in cat2searches.items():
    _df = df[df.search.isin(v)]
    random.seed(303)
    sample = _df.fn_input.unique()
    random.shuffle(sample)
    sample = sample[:700]

    cat2df[k] = _df[_df.fn_input.isin(sample)]

# _df = df[~df.search.isin(trends.search)]
# random.seed(303)
# sample = _df.fn_input.unique()
# random.shuffle(sample)
# sample = sample[:700]
# cat2df['mix'] = _df[_df.fn_input.isin(sample)]
    
cat2df['All'] = pd.concat(_df for _df in cat2df.values())

# _df = real
# random.seed(303)
# sample = _df.search.unique()
# random.shuffle(sample)
# sample = sample[:700]
# cat2df['real'] = _df[_df.search.isin(sample)]

In [21]:
cat2pixels = {}
for cat, _df in cat2df.items():

    pixels = {
        'q1_area' : [],
        'area_page' : []
    }
    for search, df_ in tqdm(_df.groupby('fn_input')):
        for col in ['q1_area', 'area_page']:
            ad = df_[df_.label == 'ads'][col].sum()
            o = df_[df_.label == 'organic'][col].sum()
            g = df_[(df_.label == 'link') | 
                    (df_.label == 'answer')][col].sum()
            amp = df_[df_.label == 'amp'][col].sum()

            e1 = g / (g + o)
            e2 = (g + ad) / (g + o + ad )
            e3 = (g + amp) / (g + o + amp)
            e4 = (g + amp + ad) / (g + o + amp + ad)
            e5 = g / (g + ad + o + amp)

            e6 =  o / (g + ad + o + amp)


            row = {
                'search_term' : search,
                'ad' : ad,
                'o' : o,
                'g' : g,
                'amp' : amp,
                'e1' : e1,
                'e2' : e2,
                'e3' : e3,
                'e4' : e4,
                'e5' : e5,
                'e6' : e6
            }

            pixels[col].append(row)

    df_pixels = pd.DataFrame(pixels['q1_area'])
    df_pixels_fp = pd.DataFrame(pixels['area_page'])
    
    cat2pixels[cat] = {
        'df_pixels' : df_pixels,
        'df_pixels_fp' : df_pixels_fp
    }

 28%|██▊       | 195/700 [00:00<00:02, 240.66it/s]<ipython-input-21-e76318c5eb8e>:16: RuntimeWarning: invalid value encountered in double_scalars
  e1 = g / (g + o)
<ipython-input-21-e76318c5eb8e>:18: RuntimeWarning: invalid value encountered in double_scalars
  e3 = (g + amp) / (g + o + amp)
 51%|█████▏    | 360/700 [00:01<00:01, 247.72it/s]<ipython-input-21-e76318c5eb8e>:17: RuntimeWarning: invalid value encountered in double_scalars
  e2 = (g + ad) / (g + o + ad )
100%|██████████| 4200/4200 [00:16<00:00, 252.82it/s]


In [22]:
cat2pixels['b']['df_pixels']

,search_term,ad,o,g,amp,e1,e2,e3,e4,e5,e6
0,../data/input/google_search/iPhone-X/2019/11/0...,0.0,221543.200000,1728.000000,0.0,0.007739,0.007739,0.007739,0.007739,0.007739,0.992261
1,../data/input/google_search/iPhone-X/2019/11/0...,0.0,0.000000,244895.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
2,../data/input/google_search/iPhone-X/2019/11/0...,0.0,971.199951,182012.065063,0.0,0.994692,0.994692,0.994692,0.994692,0.994692,0.005308
3,../data/input/google_search/iPhone-X/2019/11/0...,0.0,147354.199951,47197.733643,0.0,0.242597,0.242597,0.242597,0.242597,0.242597,0.757403
4,../data/input/google_search/iPhone-X/2019/11/0...,0.0,17974.199951,183389.085236,0.0,0.910737,0.910737,0.910737,0.910737,0.910737,0.089263
...,...,...,...,...,...,...,...,...,...,...,...
695,../data/input/google_search/iPhone-X/2020/01/0...,0.0,32895.600000,145161.000000,0.0,0.815252,0.815252,0.815252,0.815252,0.815252,0.184748
696,../data/input/google_search/iPhone-X/2020/01/0...,0.0,0.000000,166436.800293,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
697,../data/input/google_search/iPhone-X/2020/01/0...,0.0,971.199951,219773.067725,0.0,0.995600,0.995600,0.995600,0.995600,0.995600,0.004400
698,../data/input/google_search/iPhone-X/2020/01/0...,0.0,153651.600000,79706.800293,0.0,0.341564,0.341564,0.341564,0.341564,0.341564,0.658436


In [23]:
def t_student_confidence_interval(data, 
                                  confidence = 0.95,
                                  scale = 100):
    n = len(data)
    m = data.mean()
    std_err = data.sem()
    h = std_err * t.ppf((1 + confidence) / 2, n - 1)
    
    s = f"{scale * m:.1f} ± {h * scale :.1f}"
    return s

In [24]:
index2human = {
    'e1' : 'EQ1: g / (g + o)',
    'e2' : 'EQ2: (g + ad) / (g + o + ad)',
    'e3' : 'EQ3: (g + amp) / (g + o + amp)',
    'e4' : 'EQ4: (g + amp + ad) / (g + o + amp + ad)',
    'e5' : 'EQ5: g / (g + o + ads + amp)',
    
    'e6' : 'EQ6: o / (g + o + ads + amp)',
}

In [25]:
from IPython.display import display, HTML

In [26]:
for cat, v in cat2pixels.items():
    print(cat)
    df_pixels = v['df_pixels']
    results = {}
    for e in [f'e{i}' for i in range(1, 7)]:
        x_goog = t_student_confidence_interval(df_pixels[e])
        x_goog_fp = t_student_confidence_interval(df_pixels_fp[e])
        results[index2human.get(e)] = {
            'above the fold' : x_goog + '%',
            'first page' : x_goog_fp+ '%'
        }
        
    display(HTML(pd.DataFrame(results).T.to_html()))

b


,above the fold,first page
EQ1: g / (g + o),59.5 ± 2.6%,45.5 ± 0.3%
EQ2: (g + ad) / (g + o + ad),63.8 ± 2.4%,46.1 ± 0.3%
EQ3: (g + amp) / (g + o + amp),62.3 ± 2.4%,51.6 ± 0.4%
EQ4: (g + amp + ad) / (g + o + amp + ad),66.4 ± 2.2%,52.1 ± 0.4%
EQ5: g / (g + o + ads + amp),51.5 ± 2.5%,39.2 ± 0.2%
EQ6: o / (g + o + ads + amp),33.6 ± 2.2%,47.9 ± 0.4%


m


,above the fold,first page
EQ1: g / (g + o),78.9 ± 2.0%,45.5 ± 0.3%
EQ2: (g + ad) / (g + o + ad),79.6 ± 2.0%,46.1 ± 0.3%
EQ3: (g + amp) / (g + o + amp),79.5 ± 2.0%,51.6 ± 0.4%
EQ4: (g + amp + ad) / (g + o + amp + ad),80.2 ± 1.9%,52.1 ± 0.4%
EQ5: g / (g + o + ads + amp),73.6 ± 2.2%,39.2 ± 0.2%
EQ6: o / (g + o + ads + amp),19.8 ± 1.9%,47.9 ± 0.4%


e


...

,above the fold,first page
EQ1: g / (g + o),74.3 ± 2.1%,45.5 ± 0.3%
EQ2: (g + ad) / (g + o + ad),74.5 ± 2.1%,46.1 ± 0.3%
EQ3: (g + amp) / (g + o + amp),78.4 ± 1.8%,51.6 ± 0.4%
EQ4: (g + amp + ad) / (g + o + amp + ad),78.6 ± 1.8%,52.1 ± 0.4%
EQ5: g / (g + o + ads + amp),64.2 ± 2.2%,39.2 ± 0.2%
EQ6: o / (g + o + ads + amp),21.4 ± 1.8%,47.9 ± 0.4%


h


,above the fold,first page
EQ1: g / (g + o),70.3 ± 2.3%,45.5 ± 0.3%
EQ2: (g + ad) / (g + o + ad),70.9 ± 2.3%,46.1 ± 0.3%
EQ3: (g + amp) / (g + o + amp),75.9 ± 1.9%,51.6 ± 0.4%
EQ4: (g + amp + ad) / (g + o + amp + ad),76.3 ± 1.9%,52.1 ± 0.4%
EQ5: g / (g + o + ads + amp),59.9 ± 2.4%,39.2 ± 0.2%
EQ6: o / (g + o + ads + amp),23.7 ± 1.9%,47.9 ± 0.4%


All


,above the fold,first page
EQ1: g / (g + o),71.0 ± 0.9%,45.5 ± 0.3%
EQ2: (g + ad) / (g + o + ad),72.6 ± 0.9%,46.1 ± 0.3%
EQ3: (g + amp) / (g + o + amp),74.1 ± 0.8%,51.6 ± 0.4%
EQ4: (g + amp + ad) / (g + o + amp + ad),75.6 ± 0.8%,52.1 ± 0.4%
EQ5: g / (g + o + ads + amp),63.1 ± 1.0%,39.2 ± 0.2%
EQ6: o / (g + o + ads + amp),24.4 ± 0.8%,47.9 ± 0.4%


In [27]:
label2publabel = {
    'link' : 'Google Product',
    'answer' : 'Google Answer',
    'organic' : 'Non-Google',
    'amp' : 'AMP',
    'ads' : 'Ads',
}

In [28]:
labels = [
    'organic',
    'amp',
    'link',
    'answer',
    'ads'
]

In [30]:
for cat, _df in cat2df.items():
    print(f"{cat2desc.get(cat, cat)} (N={_df.fn_input.nunique():,})")
    total_area = _df['q1_area'].sum()
    total_area_fp = _df['area_page'].sum()
    n_pages = _df.fn_input.nunique()
    data = {}
    for label in labels:
        __df = _df[_df.label == label]
        area = (__df['q1_area'].sum() / total_area)
        area_page = (__df['area_page'].sum() / total_area_fp)
        freq = __df[__df['q1_area'] != 0].fn_input.nunique() / n_pages
        freq_page = __df[__df['area_page'] != 0].fn_input.nunique() /n_pages
        data[label2publabel.get(label,label)] = {
            ('Above the fold', 'area') : area,
            ('Above the fold', 'freq') : freq,
            ('First page', 'area') : area_page,
            ('First page', 'freq') : freq_page,
        }

    breakdown = pd.DataFrame(data).T
    breakdown.columns = pd.MultiIndex.from_tuples([   
        ('Above the fold', 'area'),
        ('Above the fold', 'freq'),
        ('First page', 'area'),
        ('First page', 'freq')
    ])
    
    print(((breakdown * 100).round(1).astype(str) + '%'))
    print("*" * 79)

Business (N=700)
               Above the fold        First page        
                         area   freq       area    freq
Non-Google              33.1%  92.0%      49.9%  100.0%
AMP                      5.1%  13.9%       8.8%   66.9%
Google Product          30.3%  91.9%      25.6%  100.0%
Google Answer           19.2%  75.4%      13.0%   99.3%
Ads                     12.3%  18.0%       2.7%   19.4%
*******************************************************************************
Health (N=700)
               Above the fold        First page        
                         area   freq       area    freq
Non-Google              19.4%  86.6%      53.8%  100.0%
AMP                      1.7%   5.1%       5.3%   53.6%
Google Product          28.6%  93.4%      20.7%  100.0%
Google Answer           43.9%  90.7%      18.9%   98.7%
Ads                      6.4%   8.6%       1.3%    9.3%
*******************************************************************************
Entertainment (N=700)
  